In [1]:
import tweepy
import json
import numpy as np
import pandas as pd

In [ ]:
!pip install tweepy==4.10.0 --q

In [2]:
print('TweePy version is:', tweepy.__version__)

TweePy version is: 4.10.0


In [3]:
def authenticate(creds_path):
    '''
    read the credentials and authenticate to Twitter 
    :param creds_path: str, containing path to credentials file
    :return: api, the authenticated API object 
    '''
    with open(creds_path, mode='r') as f: creds = json.load(f)
    C_KEY, C_SECRET = creds['api_key'], creds['api_key_secret']
    A_TOKEN, A_TOKEN_SECRET = creds['access_token'], creds['access_token_secret']
    auth = tweepy.OAuthHandler(consumer_key=C_KEY, consumer_secret=C_SECRET)
    auth.set_access_token(key=A_TOKEN, secret=A_TOKEN_SECRET)
    api = tweepy.API(auth=auth, wait_on_rate_limit=True)
    return api

In [110]:
USR_NAME = 'MariusCrypt0'

api = authenticate(creds_path='/content/twitter_creds.json')
timeline = api.user_timeline(screen_name=USR_NAME, count=None, exclude_replies=True, include_rts=True)

In [114]:
timeline_df = pd.DataFrame(data=[[len(tweet.text), tweet.text, tweet.created_at, tweet.user.screen_name] for tweet in timeline], columns = ['Tweet_Length', 'Tweet_Text', 'Tweet_Date', 'UserName'])
timeline_df.head(10)

,Tweet_Length,Tweet_Text,Tweet_Date,UserName
0,140,$PRIMAL just launched a huge AI crypto product...,2023-03-01 20:09:08+00:00,MariusCrypt0
1,130,$VELA just added their first new assets $MATIC...,2023-03-01 19:19:05+00:00,MariusCrypt0
2,72,$BAG will be our first 100x this year.\n\nDeli...,2023-03-01 18:46:02+00:00,MariusCrypt0
3,139,"Ok, I've crunched the numbers on $BAG and they...",2023-03-01 17:21:59+00:00,MariusCrypt0
4,34,"Follow the money, follow the $BAG.",2023-03-01 11:23:42+00:00,MariusCrypt0
5,140,$SFUND extended the lottery on their huge NFT ...,2023-03-01 10:14:48+00:00,MariusCrypt0
6,139,RT @LouisCooper_: Over $27 billion in $ETH wil...,2023-03-01 05:06:15+00:00,MariusCrypt0


In [116]:
tmp = timeline[3]

In [117]:
tweet_time = tmp.created_at.strftime('%a, %d/%B/%Y, %H:%M %Z')
print(tweet_time)

Wed, 01/March/2023, 17:21 UTC


In [123]:
import urllib
from PIL import Image

img = urllib.request.urlopen(tmp.entities['urls'][0]['expanded_url'] + '/photo/1')
img = np.asanyarray(bytearray(img.read()), dtype='uint8')

In [126]:
tmp.entities['urls'][0]['expanded_url'] + '/photo/1'

'https://twitter.com/i/web/status/1630981723772026883/photo/1'

In [124]:
Image.fromarray(img, 'RGB')

ValueError: ignored